In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Support Vector Machine

The goal of the support vector machine algorithm is to find a hyperplane in an N-dimensional space(N — the number of features) that distinctly classifies the data points.
The nearby datapoints are called Support Vectors, hence the name support vector machine.
Our objective is to find a plane that has the maximum margin, i.e the maximum distance between data points of both classes. 

![title](svm.png)

The gamma parameter defines how far the influence of a single training example reaches. 

High Gamma will consider only points close to the plausible hyperplane.

Low Gamma will consider points at greater distance.

Low regularization (C): Small C makes the cost of misclassificaiton low ("soft margin"), thus allowing more of them for the sake of wider "cushion".

High regularization (C): Large C makes the cost of misclassification high ('hard margin"), thus forcing the algorithm to explain the input data stricter and potentially overfit.

It is also possible to perform transformations on the existing features, for example: $z = x^2 + y^2$, intentionally adding another dimension. This transformation is called kernel. The kernel function is what is applied on each data instance to map the __original non-linear__ observations into a __higher-dimensional space__ in which they become __separable linearly__. More about kernels on https://stackoverflow.com/questions/33778297/support-vector-machine-kernel-types.

SVMs are inherently two-class classifiers. In particular, the most common technique in practice has been to build $N$ one-versus-rest classifiers (commonly referred to as one-versus-all or OVA classification), and to choose the class which classifies the test datum with greatest margin. Another strategy is to build a set of one-versus-one classifiers, and to choose the class that is selected by the most classifiers. While this involves building $N(N-1)/2$ classifiers, the time for training classifiers may actually decrease, since the training data set for each classifier is much smaller.

__Advantage__: when using SVM we don't need all the datapoints, only the Support Vectors, thus decreasing the complexity and time needed.

__Example description__: Guess the number shown in the image based on the pixels.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

digits = load_digits()
dir(digits)
plt.gray();
plt.imshow(digits.images[0])

X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size = 0.2)

model = SVC(C=1.0, kernel='linear')
model.fit(X_train, y_train)
model.score(X_test, y_test)

y_predicted = model.predict(X_test)
cm = confusion_matrix(y_test, y_predicted)

plt.figure(figsize=(10,7));
sns.heatmap(cm, annot = True);
# The Linear SVM algorithm presented a slightly better peformance than the Logistic Regression algorithm.

__Math behind SVM__

__Support Vector Machine is a Convex Optimization Problem__

First, we define two hyperplanes which are the boundaries for each class group (y=1 or y=-1). The two hyperplanes are described by __W__ and b as following:

\begin{equation}
W \cdot X_+ + b >= 1 
\end{equation}

\begin{equation}
W \cdot X_{-} + b <= -1
\end{equation}

Multiplying these two equations by y we get the constraint:

\begin{equation}
y \cdot (W \cdot X + b) - 1 => 0
\end{equation}

And the __separating hyperplane__ is given by:

$$ W \cdot X + b = 0 $$

Where W is perpendicular to the hyperplane.

__To find the width__ of the margin, we can do as follows:

\begin{equation}
(xb - xr) \cdot \frac{W}{|W|}
\end{equation}

Where xb and xr are vectors lying on the boundary, therefore the above equation yields in:

Width $ = \frac{2}{|W|} $, and in order to maximize the width we should minimize |W|. However, for mathematical convenience we say that we want to minimize $$ \frac{1}{2} ||W||^2 $$

From there we use Lagrange multipliers to obtain the values of __W__  and b that maximize the margin and satisfy the constraints.

$ L = \frac{1}{2}||w||^2 - \sum \alpha_i [y_i(w \cdot x + b)-1] $

Deriving with respect to w and b and equaling it to zero, we get:

$ w = \sum \alpha_i y_i x_i $   and   $ \sum \alpha_i y_i = 0 $

Thus, 

$ L = \sum{} \alpha_i - \frac{1}{2} \sum \sum \alpha_i \alpha_j y_i y_j x_i \cdot x_j $

More on: https://towardsdatascience.com/mathematics-behind-svm-support-vector-machines-84742ddda0ca

In [ ]:
#--------- SVM from scracth -----------------#
import matplotlib.pyplot as plt
import numpy as np
from sklearn.externals.joblib import parallel_backend

class Support_Vector_Machine:
    def __init__(self, visualization = True):
        self.visualization = visualization
        self.colors = {1:'r',-1:'b'}
        if self.visualization:
            self.fig = plt.figure()
            self.ax = self.fig.add_subplot(1,1,1)
    
    def fit(self, data):
        self.data = data
        opt_dict = {}
        transforms = [[1,1],
                      [-1,1],
                      [-1,-1],
                      [1,-1]]
        
        all_data = []            # Assuming a dict will be passed 
        for yi in self.data:
            for feature_set in self.data[yi]:
                for feature in feature_set:
                    all_data.append(feature)
        
        self.max_feature_value = max(all_data)
        self.min_feature_value = min(all_data)
        all_data = None
        
        step_sizes = [self.max_feature_value * 0.1,
                      self.max_feature_value * 0.01,
                      self.max_feature_value * 0.001,                   
                     ]
        
        b_range_multiple = 5
        # we dont need to take as small of steps
        # with b as we do with w
        b_multiple = 5
        latest_optimum = [self.max_feature_value*10, self.max_feature_value*10]
        
        for step in step_sizes:
            w = np.array([latest_optimum[0], latest_optimum[1]]).astype(float)
            for w[0] in np.arange(latest_optimum[0],0,-step):
                for w[1] in np.arange(latest_optimum[1],0, -step):
                    for b in np.arange(-1*(self.max_feature_value*b_range_multiple),
                                       self.max_feature_value*b_range_multiple,
                                       step*b_multiple):
                        for tr in transforms:
                            w_t = w*tr
                            found_option = True
                            # yi(xi.w+b) >= 1
                            for i in self.data:
                                for xi in self.data[i]:
                                    yi = i
                                    if not yi*(np.dot(w_t,xi)+b) >= 1:  # This condition should be respected for all
                                        found_option = False            # data points
                                        break
                                else:
                                    continue
                                break

                            if found_option:
                                opt_dict[np.linalg.norm(w_t)] = [w_t,b]
                 
            norms = sorted([n for n in opt_dict])
            opt_choice = opt_dict[norms[0]] # Get the values of opt_dict for the optimal w
            # opt_w: [w,b]
            self.w = opt_choice[0]
            self.b = opt_choice[1]
            latest_optimum[0] = abs(opt_choice[0][0]) + 2*step
            latest_optimum[1] = abs(opt_choice[0][1]) + 2*step

    def predict(self, features):
        # sign (x.w + b)
        classification = np.sign(np.dot(np.array(features), self.w) + self.b)
        if classification !=0 and self.visualization:
                self.ax.scatter(features[0], features[1], s=200, marker='*', c=self.colors[classification])
        return classification
    
    def visualize(self):
        [[self.ax.scatter(x[0],x[1], s=100, color = self.colors[i]) for x in data_dict[i]] for i in data_dict]
        
        # v = x.w + b
        def hyperplane(x0,w,b,v):            # Returns the SV after calculating the 2nd
            x1 = (-w[0]*x0 - b + v)/ w[1]    # coordinate (1st is given)
            return x0,x1
    
        datarange = (self.min_feature_value*0.9, self.max_feature_value*1.1)
        hyp_x_min = datarange[0]
        hyp_x_max = datarange[1]
        
        # positive support vectors
        psv1 = hyperplane(hyp_x_min, self.w, self.b, 1)
        psv2 = hyperplane(hyp_x_max, self.w, self.b, 1)
        self.ax.plot([psv1[0],psv2[0]],[psv1[1],psv2[1]])
        
        #negative support vectors
        nsv1 = hyperplane(hyp_x_min, self.w, self.b, -1)
        nsv2 = hyperplane(hyp_x_max, self.w, self.b, -1)
        self.ax.plot([nsv1[0],nsv2[0]],[nsv1[1],nsv2[1]])
        
        # hyperplane
        hp1 = hyperplane(hyp_x_min, self.w, self.b, 0)
        hp2 = hyperplane(hyp_x_max, self.w, self.b, 0)
        self.ax.plot([hp1[0],hp2[0]],[hp1[1],hp2[1]])


data_dict = {-1:np.array([[2,7],
                         [3,8],
                         [2.5,6]]),
              1:np.array([[7,1],
                          [6,-1],
                          [5,1]])
            }

svm = Support_Vector_Machine()
svm.fit(data=data_dict)
svm.visualize()

predict_points = [[1,1],[5,1],[6,5],[9,1],[1,9]]

#for p in predict_points:
#    svm.predict(p)